##  마케팅 활용을 위한 판매 데이터 분석

- 연령대에 따라서 좋아하는 네일 디자인이 다름
- 연령대 별, 그중에서도 최근 유입되는 신규고객에게 인기있는 상품을 찾아서 콘텐츠 제작에 활용


<실데이터이기때문에 데이터파일은 제공하지 않음>

In [25]:
import pandas as pd
import numpy as np

In [26]:
# s20 : 2020-01-01 ~ 2020-06-30 주문, s07: 2020-07-01 ~ 2020-07-08 주문
s20=pd.read_csv("../../data/sales/2020.csv",encoding='ms949')
s07=pd.read_csv("../../data/sales/200708.csv",encoding='ms949')

In [27]:
# 주문 테이블 
s20=pd.concat([s20,s07])

In [28]:
# 고객 정보 테이블
user=pd.read_csv("../../data/user/member_20200709.csv",encoding='ms949')

## DataCleansing

1) 고객 테이블: 아이디, 나이, 가입일자

2) 주문 테이블: 결제일 -> 결제월 추출 

- data 기준: 2020년 5월,6월,7월 

In [29]:
# user 아이디 포함하여 =" " 이렇게 데이터 들어와서 주문내역에 있는 아이디와 일치 시키기 위해 가공 
user['아이디']=user['아이디'].apply(lambda x: x[2:len(x)-1])

- 고객이 입력한 나이를 기반으로 연령대 구분
- 광고 마케팅 담당자가 요청한 구분값을 기준으로 함


In [30]:
user.loc[user['나이'] == '확인불가','나이'] = -1
age=user['나이'].astype('int')
user.loc[(age >=15) & (age <=25),'연령대'] = '15~25'
user.loc[(age >=26) & (age <=30),'연령대'] = '26~30'
user.loc[(age >=31) & (age <=40),'연령대'] = '31~40'
user.loc[(age >=41) & (age <=70),'연령대'] = '41~70'

In [31]:
#가입일 형식 확인
user['가입일'].head(1)

0    2020/07/09 (10:59:06)
Name: 가입일, dtype: object

In [32]:
user=user[user['가입일'] >= '2020/05/01']

In [33]:
user['가입일'] = pd.to_datetime(user['가입일'], format='%Y/%m/%d (%H:%M:%S)')

In [34]:
user['가입월']=user['가입일'].dt.month
user['가입월'].unique()

array([7, 6, 5], dtype=int64)

- 결제상태 완료인 주문 추출
- 5월~7월 주문데이터 추출
- 날짜가 str 형식으로 입력되어 있어서 데이터 형식 변경
- pd.to_datetime(), dt.month, dt.day, dt.hour, dt.year 

In [35]:
s20=s20[s20['결제상태'].str.contains('입금완료')]

In [36]:
s20['입금확인일(결제일)']=pd.to_datetime(s20['입금확인일(결제일)'])
s20['결제월']=s20['입금확인일(결제일)'].dt.month

month_list=[5,6,7]
s20=s20[s20['결제월'].isin(month_list)]
s20['결제월'].unique()

array([5, 6, 7], dtype=int64)

- 옵션 선택하는 주문의 경우 구매 상품의 모델명이 옵션 관리코드 항목으로 기입됨
- 같은 상품이므로 모델명으로 통일

In [37]:
s20.loc[s20['모델명'].isnull(),'모델명'] = s20.loc[s20['모델명'].isnull(),'옵션 관리코드']

## Table Merge

- 주문 Table 에 고객 Table Merge 하여 주문 고객이 신규 고객인지, 기존 고객인지 분류

In [38]:
user_info=user[['아이디','가입월','연령대']]

In [39]:
user_order_info=s20.merge(user_info,left_on='주문자 ID',right_on='아이디',how='left')

1) 가입월 == 결제월 -> 신규 고객 주문

2) 가입월 < 결제월 -> 기존 고객 주문

3) 가입월.isnull() -> 기존 고객 주문

* 가입월에 결제한 주문을 신규 고객 주문으로 보기로했음 > 31일에 가입하고 1일에 주문한 사람도 신규 주문으로 보게된다는 점 감안


(+) 다른 방법 

- 주문일 - 가입일 > 31 이면 신규 고객 주문으로 보기 (오차가 없어야 하는 중요한 데이터에는 이렇게 보는 것으로)
- 가입 이후 첫 주문인 고객 신규 고객 주문으로 보기 (최근 광고를 보고 가입하여 구매한 고객이 궁금한 것이기 때문에 pass)

In [40]:
join_date=user_order_info['가입월']
order_date=user_order_info['결제월']

user_order_info.loc[join_date == order_date,'신규 주문 여부'] = True
user_order_info.loc[join_date < order_date,'신규 주문 여부'] = False
user_order_info.loc[join_date.isnull(),'신규 주문 여부'] = False

### 신규 & 기존 쪼개기

In [51]:
new_user_order=user_order_info[user_order_info['신규 주문 여부'] == True]
re_user_order=user_order_info[user_order_info['신규 주문 여부'] == False]

### pivot_table 로 정리

- 각 월별 & 연령대 별로 구매한 모델 수량 합계 
- 신규 & 기존 각각 구하며 테이블을 csv 형태로 추출하여 엑셀에서 마무리하여 자료 전달 

In [63]:
new_user_rank=new_user_order.pivot_table(index=['모델명'],columns=['결제월','연령대'],values='주문품목 수량',aggfunc='sum',margins=True)
re_user_rank=re_user_order.pivot_table(index=['모델명'],columns=['결제월','연령대'],values='주문품목 수량',aggfunc='sum',margins=True)

In [64]:
new_user_rank.to_csv("../../result/0709_new_user_rank.csv",encoding='ms949')
re_user_rank.to_csv("../../result/0709_re_user_rank.csv",encoding='ms949')